## model ref :

- Classifier :
http://scikitlearn.org/stable/auto_examples/classification/plot_classifier_comparison.html

=========================================================

In [2]:
# Load basic library 

import pandas as pd, numpy as np
%matplotlib inline
%pylab inline
import seaborn  as sns 
import pylab as pl
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


# * 0 ) Data Preprocess 

In [45]:
df = pd.read_csv("/Users/yennanliu/Kaggle.com_mini_project/Speed_Dating/Speed_Dating_Data.csv", encoding="ISO-8859-1")
#input_vars = ['attr', 'sinc', 'intel', 'fun', 'amb', 'shar', 'like', 'prob']

In [46]:
df.head()

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
4,1,1.0,0,1,1,1,10,7,NaN,7,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN


In [47]:
shape(df)

(8378, 195)

In [48]:
### ONLY USING SPECIFIC FEATURES 

use_features = ['iid', 'gender', 'wave', 'pid', 'match', 'samerace', 'age_o', 'race_o', \
                 'pf_o_att', 'pf_o_sin', 'pf_o_int','pf_o_fun', 'pf_o_amb', 'pf_o_sha',\
                 'age', 'field_cd', 'race', 'imprace', 'imprelig', 'goal', 'date', 'go_out', 'sports',\
                 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing',\
                 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga', 'exphappy',\
                 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1', 'attr2_1', 'sinc2_1',\
                 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1', 'attr3_1', 'sinc3_1','fun3_1', 'intel3_1', 'amb3_1']

df_ = df[use_features]


In [49]:
df_.head(3)

,iid,gender,wave,pid,match,samerace,age_o,race_o,pf_o_att,pf_o_sin,...,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1
0,1,0,1,11.0,0,0,27.0,2.0,35.0,20.0,...,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0
1,1,0,1,12.0,0,0,22.0,2.0,60.0,0.0,...,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0
2,1,0,1,13.0,1,1,22.0,4.0,19.0,18.0,...,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0


In [50]:
# only keep rows which have non-null data 
# https://stackoverflow.com/questions/36370839/better-way-to-drop-nan-rows-in-pandas

df_ = df_.dropna(how='any')

## 0-1) help function 

In [70]:

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, mean_absolute_error
from sklearn.model_selection import cross_val_score

def train_test_separate(X,y):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)
    return X_train, X_test, y_train, y_test


# * 1) ML 

### 1-0)  input, output values define

In [58]:
# X : inpurt feature
# y : output value

X = df_.drop('match', axis=1)
y = df_[['match']]

In [67]:
X_train, X_test, y_train, y_test = train_test_separate(X,y)

### 1-1) Support Vector Classifier

In [90]:
from sklearn.svm import SVC

clf_svc = DecisionTreeClassifier(random_state=0)
clf_svc.fit(X_train, y_train)
clf_svc.score(X_test,y_test)


0.7314641744548287

### 1-2) Decision Tree Classifier

In [79]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test,y_test)


0.7314641744548287

### 1-3) Random Forest Classifier

In [81]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier(random_state=0)
clf_rf.fit(X_train, y_train)
clf_rf.score(X_test,y_test)


//anaconda/envs/g_dash/lib/python3.4/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


0.81993769470404987

### 1-4) K Neighbors Classifier

In [83]:
from sklearn.neighbors import KNeighborsClassifier

clf_kn = KNeighborsClassifier()
clf_kn.fit(X_train, y_train)
clf_kn.score(X_test,y_test)

//anaconda/envs/g_dash/lib/python3.4/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  after removing the cwd from sys.path.


0.80747663551401871

### 1-5) neural network Classifier

In [89]:
from sklearn.neural_network import MLPClassifier

clf_nn = MLPClassifier()
clf_nn.fit(X_train, y_train)
clf_nn.score(X_test,y_test)


//anaconda/envs/g_dash/lib/python3.4/site-packages/sklearn/neural_network/multilayer_perceptron.py:904: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.82616822429906545

### 1-x) XGBClassifier

In [87]:
#import xgboost as xgb

